In [ ]:
import scanpy as sc
import scanpy.external as sce

import pandas as pd
import numpy as np

import os

import triku as tk

import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
# To print versions of imports 

import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

excludes = ['builtins', 'types', 'sys']

imported_modules = [module for module in imports() if module not in excludes]

clean_modules = []

for module in imported_modules:

    sep = '.'  # to handle 'matplotlib.pyplot' cases
    rest = module.split(sep, 1)[0]
    clean_modules.append(rest)

changed_imported_modules = list(set(clean_modules))  # drop duplicates

pip_modules = !pip freeze  # you could also use `!conda list` with anaconda

for module in pip_modules:
    try:
        name, version = module.split('==')
        if name in changed_imported_modules:
            print(name + '\t' + version)
    except:
        pass

In [ ]:
seed = 0

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 150

## Reynolds et al. 2020

In [ ]:
os.getcwd()

In [ ]:
reynolds_dir = 'reynolds_2020'
os.makedirs(reynolds_dir, exist_ok=True)

### Direct h5ad download

In [ ]:
#!aria2c -x 16 https://zenodo.org/record/4536165/files/submission_210120.h5ad -d {reynolds_dir} -o reynolds_2020.h5ad

In [ ]:
adata_reynolds = sc.read_h5ad(reynolds_dir+'/reynolds_2020.h5ad')

In [ ]:
adata_reynolds_fb = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['F1', 'F2', 'F3'])]

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb, min_counts=50)

In [ ]:
sc.pp.log1p(adata_reynolds_fb)
sc.pp.normalize_total(adata_reynolds_fb)

In [ ]:
adata_reynolds_fb_healthy = adata_reynolds_fb[adata_reynolds_fb.obs['Status'] == 'Healthy']

In [ ]:
df_batches = pd.DataFrame(np.unique(adata_reynolds_fb_healthy.obs['sample_id'], return_counts=True)).transpose()

In [ ]:
df_batches.sort_values(by=1, ascending=False)

In [ ]:
selected_samples = df_batches[df_batches[1] > 50][0].values

In [ ]:
adata_reynolds_fb_healthy = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(selected_samples)]  #selected_samples)]

In [ ]:
adata_reynolds_fb_healthy

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_healthy, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_healthy, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_reynolds_fb_healthy, metric='angular', batch_key='sample_id')
tk.tl.triku(adata_reynolds_fb_healthy, n_procs=1, random_state=seed, use_adata_knn=True)

In [ ]:
sc.tl.umap(adata_reynolds_fb_healthy, min_dist=0.1, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_healthy, resolution=1.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['DPEP1', 'POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

## 4820STDY7388991

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7388991 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7388991'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7388991

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['TNMD', 'POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
os.listdir('reynolds_2020')

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom = sc.read_loom('reynolds_2020/reynolds_2020_0_4820STDY7388991_s1_dermis_fibroblasts.loom')
adata_reynolds_fb_4820STDY7388991_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7388991_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_A_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7388991.obs_names & adata_reynolds_fb_4820STDY7388991_loom.obs_names] = adata_reynolds_fb_4820STDY7388991[adata_reynolds_fb_4820STDY7388991.obs_names & adata_reynolds_fb_4820STDY7388991_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7388991_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7388991_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7388991_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7388991_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7388991_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7388991_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991_loom, resolution=1.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom, color=['LUM', 'PDGFRA', 'VIM', 'DCN', 'COL1A1', 'SFRP2', 'APOE', 'POSTN'], legend_loc='on data', cmap=magma)

0, 16, 14, 15 - Pericytes

17 - Erithrocytes

Rest - Fbs

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388991_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7388991_loom)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388991_loom, groupby='leiden', groups=['8'], reference='rest')
sc.pl.rank_genes_groups_tracksplot(adata_reynolds_fb_4820STDY7388991_loom, n_genes=100, dendrogram=False)

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom_fb = adata_reynolds_fb_4820STDY7388991_loom[~ adata_reynolds_fb_4820STDY7388991_loom.obs['leiden'].isin(['0', '14', '15', '16', '17'])]

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991_loom_fb, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991_loom_fb, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991_loom_fb, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991_loom_fb, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, min_dist=0.2, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991_loom_fb, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1'], cmap=magma, use_raw=False, legend_loc='on data')

0: Peri remnants

0 1 2 5 6 7 13: CMSS1 PRDX6 COMP GADD45A CEMIP ANTXR1 GSTO1 PID1 PHLDB2 TBX3 PGK1 PBNIP3L SEC61G TNFAIP6 BNIP3P1 SLC39A14 ENO1 BNIP3 GBE1 SLC16A3 CP ERO1A HILPDA ENO2 STC1 HIF1A
* BNIP3 is a **Bcl-2 ligand** -> *Apoptosis*
* ENO1, ENO2, GBE1, PGK1 are **glucolisis enzymes**
* These two are aasociated to HIF-1 transcriptional activity in hypoxia.
* From those cells ->
    * 1 13: SLPI+ PI16+ (A1)
    * 0 2 6 7: COL18+ COMP+ (A2)
    * 5: CD82 EDNRB PSMB3 CCL19  (B2)

3 / 5 (B2): IGFBP7 APOE C3 CXCL12 [NOT IN 5!!!] CCDC80 MGP HSPA1A CCL2 CXCL2 CCL19!!!
4 (A1 + A3): CXCL2 MGP HSPA1B 
12 (A2): COL18A1 COL16A1 COL23A1

In [ ]:
sc.pl.rank_genes_groups_tracksplot(adata_reynolds_fb_4820STDY7388991_loom_fb, dendrogram=False, n_genes=50)

## 4820STDY7388999

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7388999 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7388999'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7388999

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['TNMD', 'POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
os.listdir('reynolds_2020')

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom = sc.read_loom('reynolds_2020/reynolds_2020_8_4820STDY7388999_s2_dermis_fibroblasts.loom')
adata_reynolds_fb_4820STDY7388999_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom.obs_names

In [ ]:
adata_reynolds_fb_4820STDY7388999.obs_names

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7388999_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('Other', index=adata_reynolds_fb_4820STDY7388999_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7388999.obs_names & adata_reynolds_fb_4820STDY7388999_loom.obs_names] = adata_reynolds_fb_4820STDY7388999[adata_reynolds_fb_4820STDY7388999.obs_names & adata_reynolds_fb_4820STDY7388999_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7388999_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7388999_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7388999_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7388999_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7388999_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7388999_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999_loom, resolution=0.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1'], cmap=magma, use_raw=False, legend_loc='on data')

0, 1, 3, 7, 5: fb (although 5 and 7 are near peri)

2: pericytes

4: APC

6: T-cell

8: CEBPB ATP8B2 PHF10 ATRX COL23A1 (A2) NCKAP1 ABCA5 NFATC4 HR LAMB2 PTK7 PTOV1 MTCO1P3 MYO9B MOV10 CLCN7 ZMYM2 MMP17 COL7A1 (A2/C1) FAM20A ULK1 RBM6 VMP1 GOLGA8A || looks like an autophagy-like fb destroying the 
ECM???

9: APC

10: ?? (endo-like)

11: epithelial-related


In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom)

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom = adata_reynolds_fb_4820STDY7388999_loom[adata_reynolds_fb_4820STDY7388999_loom.obs['leiden'].isin(['0', '1', '3', '5', '7', '8'])]

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999_loom, resolution=0.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'ENO1', 'ENO2', 'BNIP3'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1'], cmap=magma, use_raw=False, legend_loc='on data')

0: HIPK2 which mediates apoptosis  || NDUFA4L2 EDNRB NR2F2 EPS8 ADAMTS9 RGS5 

1, 2, 5, 6, 8, 13 (4): TNFAIP6 PTGES 

3, 9, 7: RPS!!! || PLAC9 COL1A2 ADIRF SFRP2 S100A4 FBLN1 SPARC CRIP1 TPPP3 APOE HSPA1A

4: RPS!!! || CD82 EDNRB IGFBP7

5: MGST1

8: DDIT4 DNAJB9 HERPUD1 VEGFA MANF 

9: TOB1 CRABP2 DYNLL1 AMD1 DNAJB1 PHLDA2 

10: HSPB1 ACTG1 CCL2 EZR RGS5

11: mitochondrial!

12: mitochondrial!

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom.obs['leiden'].cat.categories

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom.obs['science_clustering'].cat.categories

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom, groupby='leiden')

In [ ]:
sc.pl.rank_genes_groups_tracksplot(adata_reynolds_fb_4820STDY7388999_loom, dendrogram=False, n_genes=75, use_raw=False)

## 4820STDY7389007

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7389007 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7389007'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7389007

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['TNMD', 'POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
os.listdir('reynolds_2020')

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom = sc.read_loom('reynolds_2020/reynolds_2020_16_4820STDY7389007_s3_dermis_fibroblasts.loom')
adata_reynolds_fb_4820STDY7389007_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom.obs_names

In [ ]:
adata_reynolds_fb_4820STDY7389007.obs_names

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7389007_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_4820STDY7389007_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7389007.obs_names & adata_reynolds_fb_4820STDY7389007_loom.obs_names] = adata_reynolds_fb_4820STDY7389007[adata_reynolds_fb_4820STDY7389007.obs_names & adata_reynolds_fb_4820STDY7389007_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7389007_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7389007_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7389007_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7389007_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7389007_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7389007_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007_loom, resolution=0.8, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

1 2 3 4 5 6 7 8 10 11 13: fibroblasts

0 9 12: pericytes

14: APC

15: kk

16: Keratinocyte

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7389007_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7389007_loom)

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom = adata_reynolds_fb_4820STDY7389007_loom[adata_reynolds_fb_4820STDY7389007_loom.obs['leiden'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '10', '11', '13'])]

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

TO DO

In [ ]:
sc.pl.rank_genes_groups_tracksplot(adata_reynolds_fb_4820STDY7389007_loom, dendrogram=False, n_genes=75)

## SKN8104899

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_SKN8104899 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['SKN8104899'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_SKN8104899

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_SKN8104899, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
os.listdir('reynolds_2020')

In [ ]:
adata_reynolds_fb_SKN8104899_loom = sc.read_loom('reynolds_2020/reynolds_2020_16_4820STDY7389007_s3_dermis_fibroblasts.loom')
adata_reynolds_fb_SKN8104899_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_SKN8104899_loom.obs_names

In [ ]:
adata_reynolds_fb_SKN8104899_loom.obs_names

In [ ]:
adata_reynolds_fb_SKN8104899_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_SKN8104899_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_SKN8104899_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_SKN8104899.obs_names & adata_reynolds_fb_SKN8104899_loom.obs_names] = adata_reynolds_fb_SKN8104899[adata_reynolds_fb_SKN8104899.obs_names & adata_reynolds_fb_SKN8104899_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_SKN8104899_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_SKN8104899_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_SKN8104899_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_SKN8104899_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_SKN8104899_loom)
sc.pp.log1p(adata_reynolds_fb_SKN8104899_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8104899_loom, resolution=0.8, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

1 2 3 4 5 6 7 8 10 11 13: fibroblasts

0 9 12: pericytes

14: APC

15: kk

16: Keratinocyte

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom)

In [ ]:
adata_reynolds_fb_SKN8104899_loom = adata_reynolds_fb_SKN8104899_loom[adata_reynolds_fb_SKN8104899_loom.obs['leiden'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '10', '11', '13'])]

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8104899_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

TO DO

In [ ]:
sc.pl.rank_genes_groups_tracksplot(adata_reynolds_fb_SKN8104899_loom, dendrogram=False, n_genes=75)

## SKN8105197

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_SKN8105197 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['SKN8105197'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_SKN8105197

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_SKN8105197, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
os.listdir('reynolds_2020')

In [ ]:
adata_reynolds_fb_SKN8105197_loom = sc.read_loom('reynolds_2020/reynolds_2020_16_4820STDY7389007_s3_dermis_fibroblasts.loom')
adata_reynolds_fb_SKN8105197_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_SKN8104899_loom.obs_names

In [ ]:
adata_reynolds_fb_SKN8105197_loom.obs_names

In [ ]:
adata_reynolds_fb_SKN8105197_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_SKN8105197_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_SKN8105197_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_SKN8105197.obs_names & adata_reynolds_fb_SKN8105197_loom.obs_names] = adata_reynolds_fb_SKN8105197[adata_reynolds_fb_SKN8105197.obs_names & adata_reynolds_fb_SKN8105197_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_SKN8105197_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_SKN8105197_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_SKN8105197_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_SKN8105197_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_SKN8105197_loom)
sc.pp.log1p(adata_reynolds_fb_SKN8105197_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8105197_loom, resolution=0.8, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

1 2 3 4 5 6 7 8 10 11 13: fibroblasts

0 9 12: pericytes

14: APC

15: kk

16: Keratinocyte

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_SKN8105197_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_SKN8105197_loom)

In [ ]:
adata_reynolds_fb_SKN8105197_loom = adata_reynolds_fb_SKN8105197_loom[adata_reynolds_fb_SKN8105197_loom.obs['leiden'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '10', '11', '13'])]

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8105197_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

TO DO

In [ ]:
sc.pl.rank_genes_groups_tracksplot(adata_reynolds_fb_SKN8105197_loom, dendrogram=False, n_genes=75)